In [ ]:
import numpy as np
import pandas as pd

from gplearn.genetic import SymbolicRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import lightgbm as lgbm

from sklearn.utils.random import check_random_state
from sklearn.preprocessing import MinMaxScaler

import plotly.graph_objects as go
from plotly.subplots import make_subplots

output_folder = './figures/'

## Data generation

In [ ]:
def target_function(x0, x1):
    return x0**2 - x1**2 + x1 + np.exp(x0) - 1
    
# Data generation
# -----------------------------------------------------
x0 = np.arange(-1, 1.1, 1/10.)
x1 = np.arange(-1, 1.1, 1/10.)
x0, x1 = np.meshgrid(x0, x1)
y_truth = target_function(x0, x1)


## Model Fitting and Visualization

In [ ]:
class ModelFitterAndVisualizer:
    def __init__(self, X_train, y_train, y_truth, scaling=False, random_state=41):
        """
        Initialize the ModelFitterAndVisualizer class with training and testing data.

        Parameters:
            X_train (pd.DataFrame): Training data features
            y_train (pd.Series): Training data target
            y_truth (pd.Series): Ground truth for predictions
            scaling (bool): Flag to indicate if scaling should be applied
            random_state (int): Seed for random number generation
        """
        self.X_train = X_train
        self.y_train = y_train
        self.y_truth = y_truth
        
        self.initialize_models(random_state)
        
        self.scaling = scaling

    # Initialize models
    # -----------------------------------------------------------------
    def initialize_models(self, random_state):
        """
        Initialize the models to be used for fitting and prediction.

        Parameters:
            random_state (int): Seed for random number generation
        """
                
        # Define kernel for GPR
        kernel = 1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0)
        
        # Define Ensemble Models Estimator
        # Decision Tree + Kernel Method
        estimators_rf_svr = [
            ('rf', RandomForestRegressor(n_estimators=30, random_state=random_state)),
            ('svr', SVR(kernel='rbf')),
        ]
        estimators_rf_gpr = [
            ('rf', RandomForestRegressor(n_estimators=30, random_state=random_state)),
            ('gpr', GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=random_state))
        ]
        # Decision Trees
        estimators_rf_xgb = [
            ('rf', RandomForestRegressor(n_estimators=30, random_state=random_state)),
            ('xgb', xgb.XGBRegressor(random_state=random_state)),
        ]
        
        self.models = [
            SymbolicRegressor(random_state=random_state),
            SVR(kernel='rbf'),
            GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=random_state),
            DecisionTreeRegressor(random_state=random_state),
            RandomForestRegressor(random_state=random_state),
            xgb.XGBRegressor(random_state=random_state),
            lgbm.LGBMRegressor(n_estimators=50, num_leaves=10, min_child_samples=3, random_state=random_state),
            VotingRegressor(estimators=estimators_rf_svr),
            StackingRegressor(estimators=estimators_rf_svr, 
                              final_estimator=RandomForestRegressor(random_state=random_state)),
            VotingRegressor(estimators=estimators_rf_gpr),
            StackingRegressor(estimators=estimators_rf_gpr, 
                              final_estimator=RandomForestRegressor(random_state=random_state)),
            VotingRegressor(estimators=estimators_rf_xgb),
            StackingRegressor(estimators=estimators_rf_xgb, 
                              final_estimator=RandomForestRegressor(random_state=random_state)),
        ]
        
        # Define graph titles
        self.titles = [
            "Ground Truth", "Training Points", 
            "SymbolicRegressor", "SVR", "GPR",
            "DecisionTree", "RForest", 
            "XGBoost", "LGBM", 
            "Vote_rf_svr", "Stack_rf_svr__rf",
            "Vote_rf_gpr", "Stack_rf_gpr__rf",
            "Vote_rf_xgb", "Stack_rf_xgb__rf",
        ] 

    def fit_models(self):
        """
        Fit the models to the training data.

        Returns:
            self: Instance of the class with fitted models
        """
        if self.scaling:
            scaler_X = MinMaxScaler()
            self.X_train_scaled = scaler_X.fit_transform(self.X_train)
        else:
            self.X_train_scaled = self.X_train.copy()
        
        for model in self.models:
            model.fit(self.X_train_scaled, self.y_train)
        return self

    def visualize_surface(self, x0, x1, width=400, height=500,
                          num_panel_columns=5,
                          vertical_spacing=0.06, horizontal_spacing=0,
                          output=None, display=False, return_fig=False):
        """
        Visualize the prediction surface for each model.

        Parameters:
            x0 (np.ndarray): Meshgrid for feature 1
            x1 (np.ndarray): Meshgrid for feature 2
            width (int): Width of the plot
            height (int): Height of the plot
            output (str): File path to save the plot
            display (bool): Flag to display the plot
        """
        
        num_plots = len(self.models) + 2
        num_panel_rows = num_plots // num_panel_columns
        
        whole_width = width * num_panel_columns
        whole_height = height * num_panel_rows

        specs = [[{'type': 'surface'} for _ in range(num_panel_columns)] for _ in range(num_panel_rows)]
        fig = make_subplots(rows=num_panel_rows, cols=num_panel_columns, 
                            specs=specs, subplot_titles=self.titles,
                            vertical_spacing=vertical_spacing, 
                            horizontal_spacing=horizontal_spacing)

        for i, model in enumerate([None, None] + self.models):
            # Assign the subplot panels
            row = i // num_panel_columns + 1
            col = i % num_panel_columns + 1
            
            # Plot training points
            if i == 1:
                fig.add_trace(go.Scatter3d(x=self.X_train[:, 0], y=self.X_train[:, 1], z=self.y_train,
										mode='markers', marker=dict(size=2, color='darkslategray'),
										name='Training Data'), row=row, col=col) 
                
                surface = go.Surface(z=self.y_truth, x=x0, y=x1, 
                                     showscale=False, opacity=.4)
                fig.add_trace(surface, row=row, col=col)
            
            # Plot predicted surface for each model and ground truth
            else:
                y_pred = self.y_truth if model is None else model.predict(np.c_[x0.ravel(), x1.ravel()]).reshape(x0.shape)
                surface = go.Surface(z=y_pred, x=x0, y=x1, 
                                     showscale=False)
                fig.add_trace(surface, row=row, col=col)
                      
        fig.update_layout(title='Model Predictions and Ground Truth', width=whole_width, height=whole_height)
        
        
        # Change camera angle
        camera = dict(
			up=dict(x=0, y=0, z=1),
			center=dict(x=0, y=0, z=0),
			eye=dict(x=-1.25, y=-1.25, z=2)
		)
        for i in range(num_plots):
            fig.update_layout(**{f'scene{i+1}_camera': camera})

                
        if display:
            fig.show()
        
        if output:
            fig.write_html(output)
            
        if return_fig:
            return fig
    

## Evaluation of Interpolation Performance

In [ ]:
rng = check_random_state(2)

# Train-test 
# -----------------------------------------------------
# Training samples
points_generated = 100
X_train = rng.uniform(-1, 1, points_generated*2).reshape(points_generated, 2)
y_train = target_function(X_train[:, 0], X_train[:, 1])

output = output_folder + 'Smooth_func_interpolate.html'
fitter_and_visualizer = ModelFitterAndVisualizer(X_train, y_train, y_truth)
fitter_and_visualizer.fit_models().visualize_surface(x0, x1, output=output)


## Evaluation of Extrapolation Performance

In [ ]:
rng = check_random_state(2)

# Train-test 
# -----------------------------------------------------
# Training samples
points_generated = 100
X_train = rng.uniform(-0.5, 1, points_generated*2).reshape(points_generated, 2)
y_train = target_function(X_train[:, 0], X_train[:, 1])

output = output_folder + 'Smooth_func_extrapolate.html'
fitter_and_visualizer = ModelFitterAndVisualizer(X_train, y_train, y_truth)
fitter_and_visualizer.fit_models().visualize_surface(x0, x1, output=output)